# Handout 9. Logistic and Poisson regressions

## 1.	Generalized linear models 
Logistic and Poisson regressions belong to the class of generalized linear models. These models are characterized by their response distribution (here the binomial distribution) and a link function, which transfers the mean value to a scale in which the relation to background variables is described as linear and additive. In a logistic regression analysis, the link function is $\mbox{logit} p = \log[p/(1− p)]$.

In R generalized linear models are handled by the `glm` function. This function is very similar to lm. The two functions use essentially the same model formulas and extractor functions (summary, etc.), but `glm` also needs to have specified which generalized linear model is desired. This is done via the family argument. To specify a binomial model with logit link (i.e., logistic regression analysis), you write `family=binomial("logit")`.

The parameters of the model can be estimated by the *method of maximum likelihood*. This is a quite general technique, similar to the least-squares method in that it finds a set of parameters that optimizes a goodness-of-fit criterion (in fact, the least-squares method itself is a slightly modified maximum-likelihood procedure). The likelihood function L(b) is simply the probability of the entire observed data set for varying parameters. 

The *deviance* is the difference between the maximized value of −2 log L and the similar quantity under a “maximal model” that fits data perfectly. Changes in deviance caused by a model reduction will be approximately chi-squared distributed with degrees of freedom equal to the change in the number of parameters.

## 2.	Logistic regression on tabular data 
Sometimes you wish to model binary outcomes, variables that can have only two possible values: diseased or nondiseased, and so forth. A linear model for transformed probabilities can be set up as
$$ 
\mbox{logit} p = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_k x_k 
$$
in which logit p = log[p/(1− p)] is the log odds. A constant additive effect on the logit scale corresponds to a constant odds ratio. The choice of the logit function is not the only one possible, but it has some mathematically convenient properties. Other choices do exist; the `probit` function (the quantile function of the normal distribution) or $\log(−\log p)$, which has a connection to survival analysis models.

One thing to notice about the logistic model is that there is no error term as in linear models. We are modelling the probability of an event directly, and that in itself will determine the variability of the binary outcome. There is no variance parameter as in the normal distribution.

In [1]:
### Construct a dataset
no.yes <- c("No","Yes")
smoking <- gl(2,1,8,no.yes)

The function `gl` is to generate levels. The first three
arguments to `gl` are, respectively, the number of levels,
the repeat count of each level, and the total length of the vector.

In [2]:
obesity <- gl(2,2,8,no.yes)
snoring <- gl(2,4,8,no.yes)
n.tot <- c(60,17,8,2,187,85,51,23)
n.hyp <- c(5,2,1,0,35,13,15,8)
data.frame(smoking,obesity,snoring,n.tot,n.hyp)

smoking,obesity,snoring,n.tot,n.hyp
No,No,No,60,5
Yes,No,No,17,2
No,Yes,No,8,1
Yes,Yes,No,2,0
No,No,Yes,187,35
Yes,No,Yes,85,13
No,Yes,Yes,51,15
Yes,Yes,Yes,23,8


R is able to fit logistic regression analyses for tabular data in two different ways. You have to specify the response as a matrix, where one column is the number of “diseased” and the other is the number of “healthy” (or “success” and “failure”, depending on context).

In [3]:
hyp.tbl <- cbind(n.hyp,n.tot-n.hyp)
glm.hyp<-glm(hyp.tbl~smoking+obesity+snoring, family=binomial("logit"))
summary(glm.hyp)


Call:
glm(formula = hyp.tbl ~ smoking + obesity + snoring, family = binomial("logit"))

Deviance Residuals: 
       1         2         3         4         5         6         7         8  
-0.04344   0.54145  -0.25476  -0.80051   0.19759  -0.46602  -0.21262   0.56231  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -2.37766    0.38018  -6.254    4e-10 ***
smokingYes  -0.06777    0.27812  -0.244   0.8075    
obesityYes   0.69531    0.28509   2.439   0.0147 *  
snoringYes   0.87194    0.39757   2.193   0.0283 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14.1259  on 7  degrees of freedom
Residual deviance:  1.6184  on 4  degrees of freedom
AIC: 34.537

Number of Fisher Scoring iterations: 4


In [4]:
anova(glm.hyp, test="Chisq")

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
NULL,NA,NA,7,14.125900,NA
smoking,1,0.002184154,6,14.123716,0.96272449
obesity,1,6.827367115,5,7.296349,0.00897715
snoring,1,5.677946593,4,1.618403,0.01717946


Notice that the Deviance column gives differences between models as variables are added to the model in turn. The deviances are approximately chi-squared distributed with the stated degrees of freedom. It is necessary to add the test="chisq" argument to get the approximate chi-square tests. *From the above, you can read that smoking is removable.*

In [5]:
glm.hyp <- glm(hyp.tbl~obesity+snoring,binomial)
anova(glm.hyp, test="Chisq")

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
NULL,NA,NA,7,14.125900,NA
obesity,1,6.825958,6,7.299942,0.008984234
snoring,1,5.621819,5,1.678123,0.017738231


## 3.	Poisson regression 
Many studies often involve the calculation of rates, typically rates of death or incidence rates of a chronic or acute disease. This is based upon counts of events occurring within a certain amount of time. The Poisson regression method is often employed for the statistical analysis of such data. However, data that are not actually counts of events but rather measurements of time until an event (or nonevent) can be analyzed by a technique which is formally equivalent.

The data that we wish to analyze can be in one of two forms. They be in aggregate form as an observed count x based on a number of person-years T. We may also have individual-level data, in which for each subject we have a time under observation Ti and a 0/1 indicator xi of whether the subject has had an event. 

The class of generalized linear models also includes the Poisson distribution, which by default uses a log link function. This is the mathematically convenient option and also a quite natural choice since it allows the linear predictor to span the entire real line. We can use this to formulate models for the log rates of the form
$$ 
\log p = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_k x_k 
$$

The following example was used by Erling B. Andersen in 1977. It involves the rates of lung cancer by age in four Danish cities and may be found as `eba1977` in the `ISwR` package.

In [6]:
require(ISwR)
names(eba1977)

Loading required package: ISwR


[1] "city"  "age"   "pop"   "cases"

In [7]:
attach(eba1977)
fit <- glm(cases~city+age, family=poisson)
summary(fit)


Call:
glm(formula = cases ~ city + age, family = poisson)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.54853  -0.57942  -0.02872   0.49797   1.68933  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  2.24374    0.20363  11.019   <2e-16 ***
cityHorsens -0.09844    0.18129  -0.543    0.587    
cityKolding -0.22706    0.18770  -1.210    0.226    
cityVejle   -0.22706    0.18770  -1.210    0.226    
age55-59    -0.03077    0.24810  -0.124    0.901    
age60-64     0.26469    0.23143   1.144    0.253    
age65-69     0.31015    0.22918   1.353    0.176    
age70-74     0.19237    0.23517   0.818    0.413    
age75+      -0.06252    0.25012  -0.250    0.803    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 27.704  on 23  degrees of freedom
Residual deviance: 20.673  on 15  degrees of freedom
AIC: 135.06

Number of Fisher Scoring ite